# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [40]:
# Obtain a new dataframe with the total quantity of each product purchased by each customer
quantity_group = data.groupby(by=['CustomerID', 'ProductName'])['Quantity'].sum()
quantity_group_df = pd.DataFrame(quantity_group)
quantity_group_df

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [80]:
# Create a product by customer matrix
data_pivot_df = pd.pivot_table(data, values='Quantity', index='ProductName', columns='CustomerID', aggfunc='sum', 
                               fill_value=0)
data_pivot_df.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


In [81]:
# Transpose the matrix
data_transpose_df = data_pivot_df.transpose()
data_transpose_df

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
200,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
264,0,0,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
412,0,0,0,0,1,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0,0,0,25,0,50,0,25,0,0,...,0,25,25,0,0,0,0,0,0,0
98069,0,0,0,25,0,25,0,0,0,25,...,0,0,0,0,0,0,0,0,0,0
98159,0,0,0,0,0,0,0,0,0,0,...,0,50,0,0,0,0,25,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [83]:
#customers_similarity = squareform(pdist(data_pivot_df, 'euclidean'))
#customers_similarity

# Calcular la matriz de distancia euclidiana entre clientes
distance_matrix = pdist(data_transpose_df, metric='euclidean')

# Convertir la matriz de distancia en una matriz cuadrada
similarity_matrix = squareform(distance_matrix)

# Crear un DataFrame con la matriz de similitud
customer_similarity_df = pd.DataFrame(similarity_matrix, index=data_transpose_df.index, 
                                      columns=data_transpose_df.index)
customer_similarity_df.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,11.916375,10.488088,11.224972,11.401754,11.090537,12.409674,11.045361,11.269428,11.489125,...,206.871941,213.180675,225.656819,198.232187,230.913404,220.501701,217.188858,228.628520,239.000000,229.773802
200,11.916375,0.000000,11.747340,12.083046,12.569805,12.288206,12.165525,12.083046,11.874342,12.000000,...,206.310446,212.635839,224.697575,197.139544,230.952376,220.202180,215.728997,228.010965,239.037654,229.704158
264,10.488088,11.747340,0.000000,11.489125,11.224972,11.445523,12.000000,11.401754,11.180340,11.747340,...,206.387984,212.946003,225.435135,197.600607,230.371439,219.136943,216.612557,228.081126,238.266657,229.773802
356,11.224972,12.083046,11.489125,0.000000,12.083046,11.789826,12.328828,11.135529,11.958261,12.165525,...,206.649462,213.082144,225.452878,197.494304,231.038958,219.952268,217.437347,228.098663,238.493186,229.464594
412,11.401754,12.569805,11.224972,12.083046,0.000000,11.704700,12.328828,11.135529,11.789826,11.747340,...,206.900942,211.679002,225.572605,197.630969,230.614397,219.733930,217.446545,227.997807,238.396728,228.927936


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [90]:
# Choose a spedific customerId. In that case, I will choose ID=33
customer_id_to_check = 33

# Get similarities for the specific customer
similarities_to_check = customer_similarity_df[customer_id_to_check]

# Sort the values and get the 5 most similar customers 
top_5_similar_customers = similarities_to_check.sort_values(ascending=False)[0:5]

# Create a dataframe with top5
top5_df = pd.DataFrame(top_5_similar_customers)
top5_df.head()

,33
CustomerID,
92255,248.594047
97324,247.024290
95017,244.276483
92492,241.642298
91777,240.547293


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [91]:
# First, it's necesary to transform the index in a new column
top5_df2 = top5_df.reset_index(drop=False)
top5_df2

,CustomerID,33
0,92255,248.594047
1,97324,247.024290
2,95017,244.276483
3,92492,241.642298
4,91777,240.547293


In [94]:
# Extract the data related to CustomerID (top5_df2) from 'quantity_group_df'
result_df = quantity_group_df.loc[top5_df2['CustomerID']]
result_df

Quantity
CustomerID ProductName                                
92255      Appetizer - Mini Egg Roll, Shrimp        24
           Appetizer - Sausage Rolls                24
           Apricots - Dried                         24
           Arizona - Green Tea                      24
           Bagel - Plain                            24
...                                                ...
91777      Whmis - Spray Bottle Trigger             24
           Wine - Blue Nun Qualitatswein            24
           Wine - White, Mosel Gold                 24
           Wine - White, Schroder And Schyl         48
           Yogurt - Blueberry, 175 Gr               24

[374 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [140]:
# Group by product name and add up the quantity of each product that the 5 customers with the highest similarity to 
# customer 33 (which was the one you selected). Add up the number of times each product was purchased for the 5 customers.
product_quantities = result_df.groupby(by=['ProductName'])['Quantity'].sum()

# Transform to dataframe
product_quantities_df = pd.DataFrame(product_quantities)

# Add index as a new column in the dataframe
product_quantities_df = product_quantities_df.reset_index(drop=False)

# Sort the dataframe by quantity of each product name
product_quantities_df = product_quantities_df.sort_values(by='Quantity', ascending=False)
product_quantities_df

,ProductName,Quantity
21,"Beef - Tenderlion, Center Cut",122
227,"Turnip - White, Organic",122
209,Squid U5 - Thailand,121
134,Muffin Chocolate Individual Wrap,98
159,Pastry - Butterscotch Baked,98
...,...,...
75,Crackers Cheez It,24
188,"Sauce - Gravy, Au Jus, Mix",24
187,Sardines,24
186,"Salsify, Organic",24


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [150]:
# Extract the records related to customer with ID '33'
customer33_df = data.loc[data['CustomerID'] == 33]
customer33_df

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
527,33,Lindsay,Santana,1899667,432,Beer - Original Organic Lager,1
1017,33,Lindsay,Santana,6509275,417,"Lamb - Pieces, Diced",1
3012,33,Lindsay,Santana,5638266,214,French Pastry - Mini Chocolate,1
3476,33,Lindsay,Santana,4280329,284,Beer - Rickards Red,1
3795,33,Lindsay,Santana,5869069,342,Mangoes,1
...,...,...,...,...,...,...,...
61648,33,Lindsay,Santana,6507033,300,"Beans - Kidney, Canned",1
62883,33,Lindsay,Santana,4150957,319,Towels - Paper / Kraft,1
63171,33,Lindsay,Santana,3056733,187,"Pepper - Black, Whole",1
66163,33,Lindsay,Santana,3179606,296,Wine - Valpolicella Masi,1


In [159]:
# Merge the products quantities dataframe with the dataframe related to customer with ID '33'
merged_df = pd.merge(product_quantities_df, customer33_df, on='ProductName', how='left', indicator=True)

# Extract in a new dataframe, the records when the merge result is 'left_only'
recommended_products_df = merged_df[merged_df['_merge']=='left_only']

# Create a new dataframe only with 'ProductName' and 'Quatity' columns.
recommended_products_df = recommended_products_df[['ProductName', 'Quantity_x']]
recommended_products_df = recommended_products_df.rename(columns={'Quantity_x': 'Quantity'})

# And after that, sort the dataframe by quatity and extract only the top-5 best-selling products.
top5_recommended_df = recommended_products_df.sort_values(by='Quantity', ascending=False)
top5_recommended_df = top5_recommended_df.head(5)
top5_recommended_df

,ProductName,Quantity
0,"Beef - Tenderlion, Center Cut",122
1,"Turnip - White, Organic",122
2,Squid U5 - Thailand,121
3,Muffin Chocolate Individual Wrap,98
4,Pastry - Butterscotch Baked,98


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [7]:
def recommendation_system(metric_type):
    # Empty dictionary to hold the recommendations for each customer
    recommendations_dict = {}
    # Create a list with CustomerIDs to iterate over
    CustomerIDs_list = data['CustomerID'].tolist()


    # STEP 1: 
    # Obtain a new dataframe with the total quantity of each product purchased by each customer
    quantity_group_df = pd.DataFrame(data.groupby(by=['CustomerID', 'ProductName'])['Quantity'].sum())


    # STEP 2: 
    # Create a product by customer matrix
    data_pivot_df = pd.pivot_table(data, values='Quantity', index='ProductName', columns='CustomerID', aggfunc='sum', 
                                   fill_value=0)
    # Transpose the matrix
    data_transpose_df = data_pivot_df.transpose()


    # STEP 3:
    # Calcular la matriz de distancia euclidiana entre clientes
    distance_matrix = pdist(data_transpose_df, metric=metric_type)
    # Convertir la matriz de distancia en una matriz cuadrada
    similarity_matrix = squareform(distance_matrix)
    # Crear un DataFrame con la matriz de similitud
    customer_similarity_df = pd.DataFrame(similarity_matrix, index=data_transpose_df.index, 
                                          columns=data_transpose_df.index)


    # STEP 4-7:
    for customer in CustomerIDs_list: #[0:length]:
        # STEP 4:
        # Get similarities for the specific customer
        similarities_to_check = customer_similarity_df[customer]
        # Sort the values and get the 5 most similar customers 
        top5_similar_customers_df = pd.DataFrame(similarities_to_check.sort_values(ascending=False)[0:5])

        # STEP 5:
        # First, it's necesary to transform the index in a new column
        top5_similar_customers_df = top5_similar_customers_df.reset_index(drop=False)
        # Extract the data related to CustomerID from 'quantity_group_df'
        top5_info_df = quantity_group_df.loc[top5_similar_customers_df['CustomerID']]

        # STEP 6:
        # Group by product name and add up the quantity of each product that the 5 customers with the highest similarity to 
        # customer 33 (which was the one you selected). Add up the number of times each product was purchased for the 5 customers.
        product_quantities_df = pd.DataFrame(top5_info_df.groupby(by=['ProductName'])['Quantity'].sum())
        # Add index as a new column in the dataframe
        product_quantities_df = product_quantities_df.reset_index(drop=False)
        # Sort the dataframe by quantity of each product name
        product_quantities_df = product_quantities_df.sort_values(by='Quantity', ascending=False)

        # STEP 7:
        # Extract the records related to customer with ID store in 'customer'
        customer_df = data.loc[data['CustomerID'] == customer]
        # Merge the products quantities dataframe with the dataframe related to customer with ID customer
        merged_df = pd.merge(product_quantities_df, customer_df, on='ProductName', how='left', indicator=True)
        # Extract in a new dataframe, the records when the merge result is 'left_only'
        recommended_products_df = merged_df[merged_df['_merge']=='left_only']
        # Create a new dataframe only with 'ProductName' and 'Quatity' columns.
        recommended_products_df = recommended_products_df[['ProductName', 'Quantity_x']]
        recommended_products_df = recommended_products_df.rename(columns={'Quantity_x': 'Quantity'})
        # And after that, sort the dataframe by quatity and extract only the top-5 best-selling products.
        top5_recommended_df = recommended_products_df.sort_values(by='Quantity', ascending=False)
        top5_recommended_df = top5_recommended_df.head(5)

        # STEP 8:
        # Append the top5 recommended to dictionary
        recommendations_dict[customer] = {
            'CustomerIDs': customer,
            'Top5Recommendations': top5_recommended_df[['ProductName']].to_dict(orient='records')
        }
        
    return recommendations_dict


In [8]:
# Use the function created below
recommendations_dict = recommendation_system('euclidean')
recommendations_dict

{61288: {'CustomerIDs': 61288,
  'Top5Recommendations': [{'ProductName': 'Bagel - Plain'},
   {'ProductName': 'Garlic - Primerba, Paste'},
   {'ProductName': 'Olive - Spread Tapenade'},
   {'ProductName': 'Squid U5 - Thailand'},
   {'ProductName': 'Table Cloth 81x81 White'}]},
 77352: {'CustomerIDs': 77352,
  'Top5Recommendations': [{'ProductName': 'Turnip - White, Organic'},
   {'ProductName': 'Whmis - Spray Bottle Trigger'},
   {'ProductName': 'Bagel - Plain'},
   {'ProductName': 'Pastry - Butterscotch Baked'},
   {'ProductName': 'Cornflakes'}]},
 40094: {'CustomerIDs': 40094,
  'Top5Recommendations': [{'ProductName': 'Turnip - White, Organic'},
   {'ProductName': 'Soup Campbells - Italian Wedding'},
   {'ProductName': 'Pastry - Butterscotch Baked'},
   {'ProductName': 'Muffin Chocolate Individual Wrap'},
   {'ProductName': 'Clam Nectar'}]},
 23548: {'CustomerIDs': 23548,
  'Top5Recommendations': [{'ProductName': 'Muffin Chocolate Individual Wrap'},
   {'ProductName': 'Turnip - White

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [27]:
# Create a function to transform de dictionary created in the below step to dataframe
def dictionary_to_dataframe(recommendations_dict):
    # Create list with columns name
    columns = ['CustomerID', 'Recommendation_1', 'Recommendation_2', 'Recommendation_3', 'Recommendation_4', 
               'Recommendation_5']
    # Create a new dataframe
    recommedations_df = pd.DataFrame(columns=columns)
    # Extract the keys in a new list
    keys_list = recommendations_dict.keys()
    
    # Use this loop to extract the CustomerID and recommendations of each customer 
    for customer_id, recommendations in recommendations_dict.items():
        # Extract the CustomerID
        row_dict = {'CustomerID': recommendations['CustomerIDs']}
        
        # Extract the 5 recommendations of each ID
        for i, recommendation in enumerate(recommendations['Top5Recommendations']):
            row_dict[f'Recommendation_{i+1}'] = recommendation['ProductName']
        
        # Transforme the dictionary in dataframe
        row_df = pd.DataFrame([row_dict])
        # Concat the new dataframe with the recommedations_df
        recommedations_df = pd.concat([recommedations_df, row_df], ignore_index=True)
        
    return recommedations_df

In [36]:
# Use the function created below
recommedations_df = dictionary_to_dataframe(recommendations_dict)
recommedations_df.head()

,CustomerID,Recommendation_1,Recommendation_2,Recommendation_3,Recommendation_4,Recommendation_5
0,61288,Bagel - Plain,"Garlic - Primerba, Paste",Olive - Spread Tapenade,Squid U5 - Thailand,Table Cloth 81x81 White
1,77352,"Turnip - White, Organic",Whmis - Spray Bottle Trigger,Bagel - Plain,Pastry - Butterscotch Baked,Cornflakes
2,40094,"Turnip - White, Organic",Soup Campbells - Italian Wedding,Pastry - Butterscotch Baked,Muffin Chocolate Individual Wrap,Clam Nectar
3,23548,Muffin Chocolate Individual Wrap,"Turnip - White, Organic","Beef - Tenderlion, Center Cut",Onion Powder,Bandage - Flexible Neon
4,78981,Spinach - Baby,Tahini Paste,Onion Powder,Cornflakes,Dried Figs


In [32]:
top_5_recommendations = recommedations_df['Recommendation_1'].value_counts().head(5)
top_5_recommendations

Recommendation_1
Turnip - White, Organic             204
Beef - Tenderlion, Center Cut       160
Soup Campbells - Italian Wedding     75
Muffin Chocolate Individual Wrap     68
Bagel - Plain                        63
Name: count, dtype: int64

In [45]:
# Function to obtain top 5 most repeated products in each column
def top5_repeated(recommedations_df):
    # Create an empty DataFrame to store the top 5 products for each column
    top5_products_by_column = pd.DataFrame()

    # Iterate over Recommendation_x columns and add top 5 to DataFrame
    for column in ['Recommendation_1', 'Recommendation_2', 'Recommendation_3', 'Recommendation_4', 'Recommendation_5']:
        top_5_recommendations = recommedations_df[column].value_counts().head(5)
        top5_products_by_column[column] = top_5_recommendations.index
        top5_products_by_column[f'{column}_Frequency'] = top_5_recommendations.values
    
    return top5_products_by_column

In [46]:
# Top 5 most repeated products in each column
top5_products = top5_repeated(recommedations_df)
top5_products.head()

,Recommendation_1,Recommendation_1_Frequency,Recommendation_2,Recommendation_2_Frequency,Recommendation_3,Recommendation_3_Frequency,Recommendation_4,Recommendation_4_Frequency,Recommendation_5,Recommendation_5_Frequency
0,"Turnip - White, Organic",204,"Turnip - White, Organic",178,Muffin Chocolate Individual Wrap,140,Muffin Chocolate Individual Wrap,214,Pastry - Butterscotch Baked,124
1,"Beef - Tenderlion, Center Cut",160,Pastry - Butterscotch Baked,95,Squid U5 - Thailand,139,Pastry - Butterscotch Baked,90,Clam Nectar,104
2,Soup Campbells - Italian Wedding,75,Muffin Chocolate Individual Wrap,76,Clam Nectar,89,Clam Nectar,69,Table Cloth 81x81 White,59
3,Muffin Chocolate Individual Wrap,68,"Beef - Tenderlion, Center Cut",60,Pastry - Butterscotch Baked,65,"Turnip - White, Organic",66,Squid U5 - Thailand,59
4,Bagel - Plain,63,Whmis - Spray Bottle Trigger,59,Bread - Hot Dog Buns,44,Bagel - Plain,52,Muffin Chocolate Individual Wrap,58


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [11]:
recommendations_dict_correlation = recommendation_system('correlation')
recommendations_dict_correlation

{61288: {'CustomerIDs': 61288,
  'Top5Recommendations': [{'ProductName': 'Onions - Cippolini'},
   {'ProductName': 'Milk - 2%'},
   {'ProductName': 'Sage - Ground'},
   {'ProductName': 'Veal - Osso Bucco'},
   {'ProductName': 'Apricots Fresh'}]},
 77352: {'CustomerIDs': 77352,
  'Top5Recommendations': [{'ProductName': 'Turkey - Oven Roast Breast'},
   {'ProductName': 'Lamb - Whole, Fresh'},
   {'ProductName': 'Water - Mineral, Natural'},
   {'ProductName': 'Tea - English Breakfast'},
   {'ProductName': 'Carbonated Water - Blackcherry'}]},
 40094: {'CustomerIDs': 40094,
  'Top5Recommendations': [{'ProductName': 'Bread - Italian Corn Meal Poly'},
   {'ProductName': 'Lamb - Pieces, Diced'},
   {'ProductName': 'Foam Cup 6 Oz'},
   {'ProductName': 'Sugar - Fine'},
   {'ProductName': 'Cookie Chocolate Chip With'}]},
 23548: {'CustomerIDs': 23548,
  'Top5Recommendations': [{'ProductName': 'Ice Cream Bar - Hageen Daz To'},
   {'ProductName': 'Mustard - Seed'},
   {'ProductName': 'Cinnamon Buns

In [37]:
recommedations_correlation_df = dictionary_to_dataframe(recommendations_dict_correlation)
recommedations_correlation_df.head()

,CustomerID,Recommendation_1,Recommendation_2,Recommendation_3,Recommendation_4,Recommendation_5
0,61288,Onions - Cippolini,Milk - 2%,Sage - Ground,Veal - Osso Bucco,Apricots Fresh
1,77352,Turkey - Oven Roast Breast,"Lamb - Whole, Fresh","Water - Mineral, Natural",Tea - English Breakfast,Carbonated Water - Blackcherry
2,40094,Bread - Italian Corn Meal Poly,"Lamb - Pieces, Diced",Foam Cup 6 Oz,Sugar - Fine,Cookie Chocolate Chip With
3,23548,Ice Cream Bar - Hageen Daz To,Mustard - Seed,Cinnamon Buns Sticky,Fondant - Icing,Juice - Orange
4,78981,Lamb - Ground,Water - Green Tea Refresher,Quiche Assorted,Olive - Spread Tapenade,Scallop - St. Jaques


In [47]:
# Top 5 most repeated products in each column
top5_products_correlation = top5_repeated(recommedations_correlation_df)
top5_products_correlation.head()

,Recommendation_1,Recommendation_1_Frequency,Recommendation_2,Recommendation_2_Frequency,Recommendation_3,Recommendation_3_Frequency,Recommendation_4,Recommendation_4_Frequency,Recommendation_5,Recommendation_5_Frequency
0,"Oil - Shortening,liqud, Fry",8,Beef - Montreal Smoked Brisket,8,Assorted Desserts,9,Assorted Desserts,8,Snapple Lemon Tea,9
1,Onion Powder,8,Wine - White Cab Sauv.on,8,Soup Campbells - Italian Wedding,9,Pasta - Angel Hair,7,"Cheese - Brie, Triple Creme",7
2,Lamb - Ground,8,Banana - Leaves,7,Smoked Paprika,8,Fondant - Icing,7,Juice - Orange,7
3,Island Oasis - Mango Daiquiri,8,Table Cloth 81x81 White,7,Cocoa Butter,8,Beef - Top Sirloin - Aaa,7,Beans - Kidney White,7
4,Grenadine,8,Soup Campbells - Italian Wedding,7,"Lamb - Pieces, Diced",7,"Cheese - Brie,danish",7,Soup Campbells - Italian Wedding,7


With the correlation algorithm there is a greater difference in the recommendations of each user than with the Euclidean algorithm, where they are repeated more frequently. 